implement a basic custom training loop in Keras.

In [1]:
import os
import warnings
import tensorflow as tf 
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Flatten, Input
from tensorflow.keras.callbacks import Callback
import numpy as np

In [4]:
# Suppress all Python warnings
warnings.filterwarnings('ignore')

# Set TensorFlow log level to suppress warnings and info messages
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# Step 1: Set Up the Environment
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data() 
x_train, x_test = x_train / 255.0, x_test / 255.0 
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(32)


### Define the model

In [5]:
model = Sequential([
    Flatten(input_shape=(28, 28)),
    Dense(128, activation='relu'),
    Dense(10)
])


### Define loss function and optimizer

- Use Sparse Categorical Crossentropy for the loss function. 
- Use the Adam optimizer. 

In [6]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True) 
optimizer = tf.keras.optimizers.Adam()

### Implement custom training loop

In [7]:
epochs = 2
# train_dataset = train_dataset.repeat(epochs)
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(32)
for epoch in range(epochs):
    print(f'Start of epoch {epoch + 1}')

    for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
        with tf.GradientTape() as tape:
            logits = model(x_batch_train, training=True)  # Forward pass
            loss_value = loss_fn(y_batch_train, logits)  # Compute loss

        # Compute gradients and update weights
        grads = tape.gradient(loss_value, model.trainable_weights)
        optimizer.apply_gradients(zip(grads, model.trainable_weights))

        # Logging the loss every 200 steps
        if step % 200 == 0:
            print(f'Epoch {epoch + 1} Step {step}: Loss = {loss_value.numpy()}')


Start of epoch 1
Epoch 1 Step 0: Loss = 2.3339295387268066
Epoch 1 Step 200: Loss = 0.4200586676597595
Epoch 1 Step 400: Loss = 0.18353486061096191
Epoch 1 Step 600: Loss = 0.16809335350990295
Epoch 1 Step 800: Loss = 0.15435132384300232
Epoch 1 Step 1000: Loss = 0.5205732583999634
Epoch 1 Step 1200: Loss = 0.18945136666297913
Epoch 1 Step 1400: Loss = 0.24824224412441254
Epoch 1 Step 1600: Loss = 0.17942407727241516
Epoch 1 Step 1800: Loss = 0.1575031876564026
Start of epoch 2
Epoch 2 Step 0: Loss = 0.05862722545862198
Epoch 2 Step 200: Loss = 0.21767151355743408
Epoch 2 Step 400: Loss = 0.12814423441886902
Epoch 2 Step 600: Loss = 0.04776795953512192
Epoch 2 Step 800: Loss = 0.0879962220788002
Epoch 2 Step 1000: Loss = 0.346660315990448
Epoch 2 Step 1200: Loss = 0.08232636004686356
Epoch 2 Step 1400: Loss = 0.15419481694698334
Epoch 2 Step 1600: Loss = 0.1385653018951416
Epoch 2 Step 1800: Loss = 0.10450499504804611


### Adding accuracy metric

In [8]:
import tensorflow as tf 
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense, Flatten 

# Step 1: Set Up the Environment
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# Normalize the pixel values to be between 0 and 1
x_train, x_test = x_train / 255.0, x_test / 255.0 

# Create a batched dataset for training
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(32)


In [9]:
model = Sequential([ 
    Flatten(input_shape=(28, 28)),  # Flatten the input to a 1D vector
    Dense(128, activation='relu'),  # First hidden layer with 128 neurons and ReLU activation
    Dense(10)  # Output layer with 10 neurons for the 10 classes (digits 0-9)
])

In [10]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)  # Loss function for multi-class classification
optimizer = tf.keras.optimizers.Adam()  # Adam optimizer for efficient training
accuracy_metric = tf.keras.metrics.SparseCategoricalAccuracy()  # Metric to track accuracy during training


### Implement loop

In [11]:
epochs = 5  # Number of epochs for training

for epoch in range(epochs):
    print(f'Start of epoch {epoch + 1}')
    
    for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
        with tf.GradientTape() as tape:
            # Forward pass: Compute predictions
            logits = model(x_batch_train, training=True)
            # Compute loss
            loss_value = loss_fn(y_batch_train, logits)
        
        # Compute gradients
        grads = tape.gradient(loss_value, model.trainable_weights)
        # Apply gradients to update model weights
        optimizer.apply_gradients(zip(grads, model.trainable_weights))
        
        # Update the accuracy metric
        accuracy_metric.update_state(y_batch_train, logits)

        # Log the loss and accuracy every 200 steps
        if step % 200 == 0:
            print(f'Epoch {epoch + 1} Step {step}: Loss = {loss_value.numpy()} Accuracy = {accuracy_metric.result().numpy()}')
    
    # Reset the metric at the end of each epoch
    accuracy_metric.reset_state()

Start of epoch 1
Epoch 1 Step 0: Loss = 2.424161434173584 Accuracy = 0.09375
Epoch 1 Step 200: Loss = 0.3653165400028229 Accuracy = 0.8387748599052429
Epoch 1 Step 400: Loss = 0.16105180978775024 Accuracy = 0.8706359267234802
Epoch 1 Step 600: Loss = 0.18409746885299683 Accuracy = 0.8850353360176086
Epoch 1 Step 800: Loss = 0.14532171189785004 Accuracy = 0.8971598148345947
Epoch 1 Step 1000: Loss = 0.44887739419937134 Accuracy = 0.9036276340484619
Epoch 1 Step 1200: Loss = 0.17933422327041626 Accuracy = 0.9098407626152039
Epoch 1 Step 1400: Loss = 0.2197543829679489 Accuracy = 0.9147260785102844
Epoch 1 Step 1600: Loss = 0.21381160616874695 Accuracy = 0.9177271723747253
Epoch 1 Step 1800: Loss = 0.17532978951931 Accuracy = 0.9216927886009216
Start of epoch 2
Epoch 2 Step 0: Loss = 0.07789589464664459 Accuracy = 1.0
Epoch 2 Step 200: Loss = 0.14208102226257324 Accuracy = 0.9597325921058655
Epoch 2 Step 400: Loss = 0.12307007610797882 Accuracy = 0.9571384191513062
Epoch 2 Step 600: Loss 